In [5]:
import glob
import os
import re
import datetime
import numpy as np
from joblib import Parallel, delayed
import xarray as xr
from netCDF4 import Dataset
import sys
import pandas as pd

import yaml
with open("config_boundary_conditions.yml", "r") as f:
    config = yaml.safe_load(f)

sys.path.insert(0, "../../")
from src.inversion_scripts.operators.operator_utilities import nearest_loc
from src.inversion_scripts.operators.TROPOMI_operator import apply_tropomi_operator
from src.inversion_scripts.utils import save_obj, load_obj

blendedTROPOMI = False#(sys.argv[1] == "True")
satelliteDir = "/n/holylfs05/LABS/jacob_lab/Lab/imi/ch4/tropomi"#sys.argv[2]
print(f"\nwrite_boundary_conditions.py output for blendedTROPOMI={blendedTROPOMI}")
print(f"Using files at {satelliteDir}")

"""
This script works in three parts, utilizing the GEOS-Chem output from run_boundary_conditions.sh
(1) Make a gridded (2.0 x 2.5 x daily) field of TROPOMI/GEOS-Chem co-locations
    - for every TROPOMI observation, apply the TROPOMI operator to the GEOS-Chem fields
    - average the TROPOMI XCH4 and GEOS-Chem XCH4 to a (2.0 x 2.5) grid for each day
(2) Make a gridded (2.0 x 2.5 x daily) field of the bias between TROPOMI and GEOS-Chem
    - subtract the TROPOMI and GEOS-Chem grids from part 1 to get a starting point for the bias
    - smooth this field spatially (5 lon grid boxes, 5 lat grid boxes) then temporally (15 days backwards)
    - fill NaN values with the latitudinal average at that time
        - for a latitudinal average to be defined, there must be >= 30 grid cells at that latitude
        - when a latitudinal average cannot be found, the closest latitudinal average is used
(3) Write the boundary conditions
    - using the bias from Part 2, subtract the (GC-TROPOMI) bias from the GC boundary conditions
"""

def get_TROPOMI_times(filename):
    
    """
    Function that parses the TROPOMI filenames to get the start and end times.
    Example input (str): S5P_RPRO_L2__CH4____20220725T152751_20220725T170921_24775_03_020400_20230201T100624.nc
    Example output (tuple): (np.datetime64('2022-07-25T15:27:51'), np.datetime64('2022-07-25T17:09:21'))
    """

    file_times = re.search(r'(\d{8}T\d{6})_(\d{8}T\d{6})', filename)
    assert file_times is not None, "check TROPOMI filename - wasn't able to find start and end times in the filename"
    start_TROPOMI_time = np.datetime64(datetime.datetime.strptime(file_times.group(1), "%Y%m%dT%H%M%S"))
    end_TROPOMI_time = np.datetime64(datetime.datetime.strptime(file_times.group(2), "%Y%m%dT%H%M%S"))
    
    return start_TROPOMI_time, end_TROPOMI_time

def apply_tropomi_operator_to_one_tropomi_file(filename):
    
    """
    Run apply_tropomi_operator from src/inversion_scripts/operators/TROPOMI_operator.py for a single TROPOMI file (then saves it to a pkl file)
    Example input (str): S5P_RPRO_L2__CH4____20220725T152751_20220725T170921_24775_03_020400_20230201T100624.nc
    Example output: write the file config["workdir"]/step1/S5P_RPRO_L2__CH4____20220725T152751_20220725T170921_24775_03_020400_20230201T100624_GCtoTROPOMI.pkl
    """
    
    result = apply_tropomi_operator(
        filename = filename,
        BlendedTROPOMI = blendedTROPOMI,
        n_elements = False, # Not relevant
        gc_startdate = start_time_of_interest,
        gc_enddate = end_time_of_interest,
        xlim = [-180, 180],
        ylim = [-90, 90],
        gc_cache = os.path.join(config["workDir"], "gc_run", "OutputDir"),
        build_jacobian = False, # Not relevant
        sensi_cache = False) # Not relevant
    
    return result["obs_GC"],filename


write_boundary_conditions.py output for blendedTROPOMI=False
Using files at /n/holylfs05/LABS/jacob_lab/Lab/imi/ch4/tropomi


In [6]:
### Part 1 ###

# From config file, get the start and end times that we will be writing boundary conditions for (+20 days on the end because of our temporal smoothing)
start_time_of_interest = np.datetime64(datetime.datetime.strptime(config["startDate"], "%Y%m%d"))
end_time_of_interest = np.datetime64(datetime.datetime.strptime(config["endDate"], "%Y%m%d"))

# List of all TROPOMI files that interesct our time period of interest
TROPOMI_files = sorted([file for file in glob.glob(os.path.join(satelliteDir, "*.nc"))
                        if (start_time_of_interest <= get_TROPOMI_times(file)[0] <= end_time_of_interest)
                        and (start_time_of_interest <= get_TROPOMI_times(file)[1] <= end_time_of_interest)])
print(f"First TROPOMI file -> {TROPOMI_files[0]}")
print(f"Last TROPOMI file  -> {TROPOMI_files[-1]}")

# Using as many cores as you have, apply the TROPOMI operator to each file
obsGC_and_filenames = Parallel(n_jobs=-1)(delayed(apply_tropomi_operator_to_one_tropomi_file)(filename) for filename in TROPOMI_files)

# Read any of the GEOS-Chem files to get the lat/lon grid
with xr.open_dataset(glob.glob(os.path.join(config["workDir"], "gc_run", "OutputDir", "GEOSChem.SpeciesConc*.nc4"))[0]) as data:
    LON = data["lon"].values
    LAT = data["lat"].values

# List of all days in our time range of interest
alldates = np.arange(start_time_of_interest, end_time_of_interest + np.timedelta64(1, 'D'), dtype='datetime64[D]')
alldates = [day.astype(datetime.datetime).strftime("%Y%m%d") for day in alldates]

# Initialize arrays for regridding
daily_TROPOMI = np.zeros((len(LON), len(LAT), len(alldates)))
daily_GC = np.zeros((len(LON), len(LAT), len(alldates)))
daily_count = np.zeros((len(LON), len(LAT), len(alldates)))

# Loop thorugh all of the files which now contain TROPOMI and the corresponding GC XCH4
for obsGC,filename in obsGC_and_filenames:
    NN = obsGC.shape[0]
    if NN == 0:
        continue

    # For each TROPOMI observation, assign it to a GEOS-Chem grid cell
    for iNN in range(NN):
            
            # Which day are we on (this is not perfect right now because orbits can cross from one day to the next...
            # but it is the best we can do right now without changing apply_tropomi_operator)
            file_times = re.search(r'(\d{8}T\d{6})_(\d{8}T\d{6})', filename)
            assert file_times is not None, "check TROPOMI filename - wasn't able to find start and end times in the filename"
            date = datetime.datetime.strptime(file_times.group(1), "%Y%m%dT%H%M%S").strftime("%Y%m%d")
            time_ind = alldates.index(date)

            c_TROPOMI, c_GC, lon0, lat0 = obsGC[iNN, :4]
            ii = nearest_loc(lon0, LON, tolerance=5)
            jj = nearest_loc(lat0, LAT, tolerance=4)
            daily_TROPOMI[ii, jj, time_ind] += c_TROPOMI
            daily_GC[ii, jj, time_ind] += c_GC
            daily_count[ii, jj, time_ind] += 1

# Normalize by how many observations got assigned to a grid cell to finish the regridding
daily_count[daily_count == 0] = np.nan
daily_TROPOMI = daily_TROPOMI / daily_count
daily_GC = daily_GC / daily_count

# Change dimensions
regrid_TROPOMI = np.einsum("ijl->lji", daily_TROPOMI) # (lon, lat, time) -> (time, lat, lon)
regrid_GC = np.einsum("ijl->lji", daily_GC) # (lon, lat, time) -> (time, lat, lon)

# Make a Dataset with variables of (TROPOMI_CH4, GC_CH4) and dims of (lon, lat, time)
ds = xr.Dataset({
    'TROPOMI_CH4': xr.DataArray(
        data = regrid_TROPOMI,
        dims = ["time", "lat", "lon"],
        coords = {"time": alldates, "lat": LAT, "lon": LON}
        ),
    'GC_CH4': xr.DataArray(
        data = regrid_GC,
        dims = ["time", "lat", "lon"],
        coords = {"time": alldates, "lat": LAT, "lon": LON}
        ),
})

### Part 2 ###

bias = ds["GC_CH4"] - ds["TROPOMI_CH4"]

# Smooth spatially
bias = bias.rolling(lat=5,              # five lat grid boxes (10 degrees)
                    lon=5,              # five lon grid boxes (12.5 degrees)
                    center=True,        # five boxes includes the one we are cented on
                    min_periods=25/2    # half of the grid cells have a value to not output NaN
                    ).mean(skipna=True)

# Smooth temporally
bias = bias.rolling(time=15,            # average 15 days back in time (including the time we are centered on)
                    min_periods=1,      # only one of the time values must have a value to not output NaN
                ).mean(skipna=True)

# Create a dataarray with latitudinal average for each time step
# We will fill the NaN values in bias with these averages
nan_value_filler_2d = bias.copy()
nan_value_filler_2d = (nan_value_filler_2d.where(nan_value_filler_2d.count("lon") >= 30) # there needs to be 30 grid boxes       
                                        .mean(dim=["lon"], skipna=True)                #    at this lat to define a mean
                                        .interpolate_na(dim="lat", method="nearest")   # fill in "middle" NaN values
                                        .bfill(dim="lat")                              # fill in NaN values towards -90 deg
                                        .ffill(dim="lat")                              # fill in NaN values towards +90 deg
                                    )

# Expand to 3 dimensions
nan_value_filler_3d = bias.copy() * np.nan
for i in range(len(LON)):
    nan_value_filler_3d[:,:,i] = nan_value_filler_2d

# Use these values to fill NaNs
bias = bias.fillna(nan_value_filler_3d)

print(f"Average bias (GC-TROPOMI): {bias.mean().values:.2f} ppb\n")

# If there are still NaNs (this will happen when TROPOMI data is missing), use 0.0 ppb as the bias but warn the user
for t in range(len(bias["time"].values)):
    if np.any(np.isnan(bias[t,:,:].values)):
        print(f"WARNING -> using 0.0 ppb as bias for {bias['time'].values[t]}")
        bias[t,:,:] = bias[t,:,:].fillna(0)

### Part 3 ###

# Get dates and convert the total column bias to mol/mol
strdate = bias["time"].values
bias = bias.values * 1e-9

# Only write BCs for our date range
files = sorted(glob.glob(os.path.join(config["workDir"], "gc_run", "OutputDir", "GEOSChem.BoundaryConditions*.nc4")))
files = [
    f
    for f in files
    if (
        (np.datetime64(re.search(r'(\d{8})_(\d{4}z)', f).group(1)) >= np.datetime64(config["startDate"]))
        & (np.datetime64(re.search(r'(\d{8})_(\d{4}z)', f).group(1)) <= np.datetime64(config["endDate"]))
    )
]

# For each file, remove the total column bias from each level of the GEOS-Chem boundary condition
for filename in files:
    l = [index for index,date in enumerate(strdate) if date == re.search(r'(\d{8})_(\d{4}z)', filename).group(1)]
    assert len(l) == 1, "ERROR -> there should only be bias per boundary condition file"
    index = l[0]
    bias_for_this_boundary_condition_file = bias[index, :, :]

    with xr.open_dataset(filename) as ds:
        original_data = ds["SpeciesBC_CH4"].values.copy()
        for t in range(original_data.shape[0]):
            for lev in range(original_data.shape[1]):
                original_data[t, lev, :, :] -= bias_for_this_boundary_condition_file
        ds["SpeciesBC_CH4"].values = original_data
        if blendedTROPOMI:
            print(f"Writing to {os.path.join(config['workDir'], 'blended-boundary-conditions', os.path.basename(filename))}")
            ds.to_netcdf(os.path.join(config["workDir"], "blended-boundary-conditions", os.path.basename(filename)))
        else:
            print(f"Writing to {os.path.join(config['workDir'], 'tropomi-boundary-conditions', os.path.basename(filename))}")
            ds.to_netcdf(os.path.join(config["workDir"], "tropomi-boundary-conditions", os.path.basename(filename)))

First TROPOMI file -> /n/holylfs05/LABS/jacob_lab/Lab/imi/ch4/tropomi/S5P_RPRO_L2__CH4____20180430T001950_20180430T020120_02818_03_020400_20221107T155110.nc
Last TROPOMI file  -> /n/holylfs05/LABS/jacob_lab/Lab/imi/ch4/tropomi/S5P_RPRO_L2__CH4____20180530T211722_20180530T225852_03256_03_020400_20221110T091912.nc


IndexError: index 2219 is out of bounds for axis 1 with size 215

In [9]:
apply_tropomi_operator_to_one_tropomi_file(TROPOMI_files[0])

IndexError: index 2219 is out of bounds for axis 1 with size 215

In [10]:
filename = TROPOMI_files[0]
BlendedTROPOMI = blendedTROPOMI
n_elements = False # Not relevant
gc_startdate = start_time_of_interest
gc_enddate = end_time_of_interest
xlim = [-180, 180]
ylim = [-90, 90]
gc_cache = os.path.join(config["workDir"], "gc_run", "OutputDir")
build_jacobian = False # Not relevant
sensi_cache = False

In [14]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime
from shapely.geometry import Polygon
from src.inversion_scripts.utils import (
    filter_tropomi,
    filter_blended,
)
from src.inversion_scripts.operators.operator_utilities import (
    get_gc_lat_lon,
    read_all_geoschem,
    merge_pressure_grids,
    remap,
    remap_sensitivities,
    get_gridcell_list,
    nearest_loc,
)

def read_tropomi(filename):
    """
    Read TROPOMI data and save important variables to dictionary.

    Arguments
        filename [str]  : TROPOMI netcdf data file to read

    Returns
        dat      [dict] : Dictionary of important variables from TROPOMI:
                            - CH4
                            - Latitude
                            - Longitude
                            - QA value
                            - UTC time
                            - Time (utc time reshaped for orbit)
                            - Averaging kernel
                            - SWIR albedo
                            - NIR albedo
                            - Blended albedo
                            - CH4 prior profile
                            - Dry air subcolumns
                            - Latitude bounds
                            - Longitude bounds
                            - Vertical pressure profile
    """

    # Initialize dictionary for TROPOMI data
    dat = {}

    # Catch read errors in any of the variables
    try:
        # Store methane, QA, lat, lon, and time
        with xr.open_dataset(filename, group="PRODUCT") as tropomi_data:
            dat["methane"] = tropomi_data["methane_mixing_ratio_bias_corrected"].values[0, :, :]
            dat["qa_value"] = tropomi_data["qa_value"].values[0, :, :]
            dat["longitude"] = tropomi_data["longitude"].values[0, :, :]
            dat["latitude"] = tropomi_data["latitude"].values[0, :, :]

            utc_str = tropomi_data["time_utc"].values[0,:]
            utc_str = np.array([d.replace("Z","") for d in utc_str]).astype("datetime64[ns]")
            dat["time"] = np.repeat(utc_str[:, np.newaxis], dat["methane"].shape[1], axis=1)

        # Store column averaging kernel, SWIR and NIR surface albedo
        with xr.open_dataset(filename, group="PRODUCT/SUPPORT_DATA/DETAILED_RESULTS") as tropomi_data:
            dat["column_AK"] = tropomi_data["column_averaging_kernel"].values[0, :, :, ::-1]
            dat["swir_albedo"] = tropomi_data["surface_albedo_SWIR"].values[0, :, :]
            dat["nir_albedo"] = tropomi_data["surface_albedo_NIR"].values[0, :, :]
            dat["blended_albedo"] = 2.4 * dat["nir_albedo"] - 1.13 * dat["swir_albedo"]

        # Store methane prior profile, dry air subcolumns
        with xr.open_dataset(filename, group="PRODUCT/SUPPORT_DATA/INPUT_DATA") as tropomi_data:
            dat["methane_profile_apriori"] = tropomi_data["methane_profile_apriori"].values[0, :, :, ::-1]  # mol m-2
            dat["dry_air_subcolumns"] = tropomi_data["dry_air_subcolumns"].values[0, :, :, ::-1]  # mol m-2
            dat["surface_classification"] = (tropomi_data["surface_classification"].values[:].astype("uint8") & 0x03).astype(int)

            # Also get pressure interval and surface pressure for use below
            pressure_interval = (tropomi_data["pressure_interval"].values[0, :, :] / 100)  # Pa -> hPa
            surface_pressure = (tropomi_data["surface_pressure"].values[0, :, :] / 100)  # Pa -> hPa

        # Store latitude and longitude bounds for pixels
        with xr.open_dataset(filename, group="PRODUCT/SUPPORT_DATA/GEOLOCATIONS") as tropomi_data:
            dat["longitude_bounds"] = tropomi_data["longitude_bounds"].values[0, :, :, :]
            dat["latitude_bounds"] = tropomi_data["latitude_bounds"].values[0, :, :, :]

        # Store vertical pressure profile
        n1 = dat["methane"].shape[0]  # length of along-track dimension (scanline) of retrieval field
        n2 = dat["methane"].shape[1]  # length of across-track dimension (ground_pixel) of retrieval field
        pressures = np.full([n1, n2, 12 + 1], np.nan, dtype=np.float32)
        for i in range(12 + 1):
            pressures[:, :, i] = surface_pressure - i * pressure_interval
        dat["pressures"] = pressures

    # Return an error if any of the variables were not read correctly
    except Exception as e:
        print(f"Error opening {filename}: {e}")
        return None

    return dat

def read_blended(filename):
    """
    Read Blended TROPOMI+GOSAT data and save important variables to dictionary.
    Arguments
        filename [str]  : Blended TROPOMI+GOSAT netcdf data file to read
    Returns
        dat      [dict] : Dictionary of important variables from Blended TROPOMI+GOSAT:
                            - CH4
                            - Latitude
                            - Longitude
                            - Time (utc time reshaped for orbit)
                            - Averaging kernel
                            - SWIR albedo
                            - NIR albedo
                            - Blended albedo
                            - CH4 prior profile
                            - Dry air subcolumns
                            - Latitude bounds
                            - Longitude bounds
                            - Surface classification
                            - Chi-Square for SWIR
                            - Vertical pressure profile
    """
    assert "BLND" in filename, f"BLND not in filename {filename}, but a blended function is being used"

    try:
        # Initialize dictionary for Blended TROPOMI+GOSAT data
        dat = {}

        # Extract data from netCDF file to our dictionary
        with xr.open_dataset(filename) as blended_data:

            dat["methane"] = blended_data["methane_mixing_ratio_blended"].values[:]
            dat["longitude"] = blended_data["longitude"].values[:]
            dat["latitude"] = blended_data["latitude"].values[:]
            dat["column_AK"] = blended_data["column_averaging_kernel"].values[:, ::-1]
            dat["swir_albedo"] = blended_data["surface_albedo_SWIR"][:]
            dat["nir_albedo"] = blended_data["surface_albedo_NIR"].values[:]
            dat["blended_albedo"] = 2.4 * dat["nir_albedo"] - 1.13 * dat["swir_albedo"]
            dat["methane_profile_apriori"] = blended_data["methane_profile_apriori"].values[:, ::-1]
            dat["dry_air_subcolumns"] = blended_data["dry_air_subcolumns"].values[:, ::-1]
            dat["longitude_bounds"] = blended_data["longitude_bounds"].values[:]
            dat["latitude_bounds"] = blended_data["latitude_bounds"].values[:]
            dat["surface_classification"] = (blended_data["surface_classification"].values[:].astype("uint8") & 0x03).astype(int)
            dat["chi_square_SWIR"] = blended_data["chi_square_SWIR"].values[:]

            # Remove "Z" from time so that numpy doesn't throw a warning
            utc_str = blended_data["time_utc"].values[:]
            dat["time"] = np.array([d.replace("Z","") for d in utc_str]).astype("datetime64[ns]")

            # Need to calculate the pressure for the 13 TROPOMI levels (12 layer edges)
            pressure_interval = (blended_data["pressure_interval"].values[:] / 100)  # Pa -> hPa
            surface_pressure = (blended_data["surface_pressure"].values[:] / 100)    # Pa -> hPa
            n = len(dat["methane"])
            pressures = np.full([n, 12 + 1], np.nan, dtype=np.float32)
            for i in range(12 + 1):
                pressures[:, i] = surface_pressure - i * pressure_interval
            dat["pressures"] = pressures

        # Add an axis here to mimic the (scanline, groundpixel) format of operational TROPOMI data
        # This is so the blended data will be compatible with the TROPOMI operators
        for key in dat.keys():
            dat[key] = np.expand_dims(dat[key], axis=0)

    except Exception as e:
        print(f"Error opening {filename}: {e}")
        return None

    return dat

In [17]:
# Read TROPOMI data
assert isinstance(BlendedTROPOMI, bool), "BlendedTROPOMI is not a bool"
if BlendedTROPOMI:
    TROPOMI = read_blended(filename)
else:
    TROPOMI = read_tropomi(filename)
if TROPOMI == None:
    print(f"Skipping {filename} due to file processing issue.")
    print("ERROR")

if BlendedTROPOMI:
    # Only going to consider blended data within lat/lon/time bounds and wihtout problematic coastal pixels
    sat_ind = filter_blended(TROPOMI, xlim, ylim, gc_startdate, gc_enddate)
else:
    # Only going to consider TROPOMI data within lat/lon/time bounds and with QA > 0.5
    sat_ind = filter_tropomi(TROPOMI, xlim, ylim, gc_startdate, gc_enddate)

# Number of TROPOMI observations
n_obs = len(sat_ind[0])
# print("Found", n_obs, "TROPOMI observations.")

# If need to build Jacobian from GEOS-Chem perturbation simulation sensitivity data:
if build_jacobian:
    # Initialize Jacobian K
    jacobian_K = np.zeros([n_obs, n_elements], dtype=np.float32)
    jacobian_K.fill(np.nan)

# Initialize a list to store the dates we want to look at
all_strdate = []

# For each TROPOMI observation
for k in range(n_obs):
    # Get the date and hour
    iSat = sat_ind[0][k]  # lat index
    jSat = sat_ind[1][k]  # lon index
    time = pd.to_datetime(str(TROPOMI["time"][iSat,jSat]))
    strdate = time.round("60min").strftime("%Y%m%d_%H")
    all_strdate.append(strdate)
all_strdate = list(set(all_strdate))

IndexError: index 2219 is out of bounds for axis 1 with size 215

In [19]:
sat_ind = filter_tropomi(TROPOMI, xlim, ylim, gc_startdate, gc_enddate)

In [27]:
TROPOMI["surface_classification"].shape

(1, 2906, 215)

In [28]:
TROPOMI["methane"].shape

(2906, 215)